In [1]:
import numpy as np
import pandas as pd
from duckreg.estimators import DuckRegression, DuckMundlak

In [2]:
m = DuckRegression(
    db_name="/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/scratch/demean_temp.db",
    table_name="read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/__modis.parquet/**/*.parquet')",
    formula="median ~ viirs_annual",
    cluster_col="country",
    n_bootstraps=0,
    seed=42,
    round_strata=5,
)
m.fit()
m.fit_vcov()
results = m.summary()
restab = pd.DataFrame(
    np.c_[results["point_estimate"], results["standard_error"]],
    columns=["point_estimate", "standard_error"],
)

RuntimeError: Query interrupted

In [ ]:
m.df_compressed#["count"].sum()

,ntl_harm,avg_ntl_harm_fe0,avg_ntl_harm_fe1,count,sum_modis_median,mean_modis_median
0,5.0,6.952381,1.763208,54058,1.559625e+07,288.509556
1,0.0,1.190476,1.628417,723864,2.065568e+08,285.353056
2,9.0,10.714286,1.628417,27608,7.878037e+06,285.353399
3,17.0,10.714286,2.056254,5626,1.629044e+06,289.556280
4,7.0,1.904762,2.056254,193714,5.575941e+07,287.843980
...,...,...,...,...,...,...
1269509,58.0,19.523810,1.649564,1,2.686800e+02,268.680000
1269510,50.0,10.857143,1.584019,2,5.747900e+02,287.395000
1269511,63.0,3.761905,2.014306,2,5.893800e+02,294.690000
1269512,39.0,56.047619,1.478400,1,2.955600e+02,295.560000


In [6]:
m.point_estimate

array([[ 2.81784632e+02],
       [-5.64303858e-03],
       [ 4.18031762e-02],
       [ 3.91785134e-01]])

In [ ]:
m = DuckMundlak(
    db_name="/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/scratch/modis.db",
    table_name="read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet')",
    outcome_var="modis_median",
    covariates=["ntl_harm"],
    fe_cols=["pixel_id","year"],
    cluster_col="country",
    n_bootstraps=999,
    round_strata=5,
    seed=42,
    duckdb_kwargs={
        "temp_directory": "/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/scratch/duckdb_swap",
        "memory_limit": "48GB",
        "max_temp_directory_size": "1024GB"
        },
    overwrite=False
)
m.fit()
m.fit_vcov()
results = m.summary()
restab = pd.DataFrame(
    np.c_[results["point_estimate"], results["standard_error"]],
    columns=["point_estimate", "standard_error"],
)

Computing averages for fixed effect dimension 1/2: pixel_id


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Computing averages for fixed effect dimension 2/2: year


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Creating design matrix with all fixed effect averages


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [31]:
restab

,point_estimate,standard_error
0,282.425524,0.000198
1,0.032956,0.000043


In [32]:
m.df_compressed['count'].sum()

np.int64(17825197929)

In [7]:
restab

,point_estimate,standard_error
0,276.744253,0.000214
1,0.188669,0.000043


----

In [1]:
import duckdb

con = duckdb.connect("/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/scratch/demeaned_data.db")

In [2]:
for key, val in {
    "temp_directory": "/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/scratch/duckdb_swap",
    "memory_limit": "48GB",
    "max_temp_directory_size": "1024GB"
    }.items():
    con.execute(f"SET {key} = '{val}'")

In [9]:
con.sql("""
        CREATE OR REPLACE TABLE design_matrix AS
            SELECT pixel_id, year, modis_median, ix, iy
            FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet')
        """)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
fe_cols = ["pixel_id", "year"]

In [ ]:
for fe_col in fe_cols:
    con.sql(f"""
    CREATE OR REPLACE TEMP TABLE m_{fe_col} AS
    SELECT {fe_col}, avg(modis_median) AS {fe_col}_avg_modis_median
        FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet')
        GROUP BY {fe_col};
    """)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
partition_cols = {
    "pixel_id": ["ix"], #, "iy"
    "year": []
}

partition_col_uniques = {col: {} for col in partition_cols.keys()}

for fe_col in partition_col_uniques.keys():
    for batch_col in partition_cols[fe_col]:
        partition_col_uniques[fe_col][batch_col] = con.sql(
            f"""
            SELECT DISTINCT {batch_col}
            FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet')
            ORDER BY {batch_col}
            """
        ).fetchnumpy()[batch_col].tolist()
        
from itertools import product
iterator = list(product(*partition_col_uniques["pixel_id"].values()))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [43]:
partition_cols = ["ix"]

partition_col_uniques = {col: [] for col in partition_cols}

for partition_col in partition_col_uniques:
    partition_col_uniques[partition_col] = con.sql(
        f"""
        SELECT DISTINCT {partition_col}
        FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet')
        ORDER BY {partition_col}
        """
    ).fetchnumpy()[partition_col].tolist()
    
from itertools import product
iterator = list(product(*partition_col_uniques.values()))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [45]:
con.sql(f"""
        CREATE OR REPLACE TABLE design_matrix AS
            SELECT pixel_id, year, modis_median, 
            FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet')
            WITH NO DATA;
        """)

In [46]:
for fe_col in fe_cols:
    con.sql(f"""
            ALTER TABLE design_matrix
                ADD COLUMN {fe_col}_avg_modis_median DOUBLE;
            """)

In [56]:
con.sql(
    """
    SELECT *
    FROM (SELECT *
    FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet')
        LEFT JOIN m_pixel_id
        USING (pixel_id))
        LEFT JOIN m_year
        USING (year)
    """
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────┬────────────────────┬────────────────────┬────────────────────┬──────────────┬──────────┬─────────┬────────────┬────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬────────────┬───────┬───────┬───────────────────────────┬───────────────────────┐
│ year  │    modis_median    │     modis_mean     │   modis_rollmax3   │ viirs_annual │ ntl_harm │ reg_fav │ nb_mines_a │ nb_diamond │ subdivision │ country │ HDI_ME  │ HDI_VH  │  WB_HI  │  WB_LM  │ HDI_HI  │  WB_UM  │  pixel_id  │  ix   │  iy   │ pixel_id_avg_modis_median │ year_avg_modis_median │
│ int16 │       double       │       double       │       double       │    double    │  double  │ boolean │   float    │   float    │    float    │  float  │ boolean │ boolean │ boolean │ boolean │ boolean │ boolean │   uint64   │ int64 │ int64 │          double           │        double         │
├───────┼────────────────────┼────────────────────┼────────────────────┼──────────────┼──────────┼──

In [55]:
con.sql(
    """
    SELECT *
    FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet')
        LEFT JOIN m_pixel_id
        USING (pixel_id)
    """
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌───────┬────────────────────┬────────────────────┬────────────────────┬──────────────┬──────────┬─────────┬────────────┬────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬────────────┬───────┬───────┬───────────────────────────┐
│ year  │    modis_median    │     modis_mean     │   modis_rollmax3   │ viirs_annual │ ntl_harm │ reg_fav │ nb_mines_a │ nb_diamond │ subdivision │ country │ HDI_ME  │ HDI_VH  │  WB_HI  │  WB_LM  │ HDI_HI  │  WB_UM  │  pixel_id  │  ix   │  iy   │ pixel_id_avg_modis_median │
│ int16 │       double       │       double       │       double       │    double    │  double  │ boolean │   float    │   float    │    float    │  float  │ boolean │ boolean │ boolean │ boolean │ boolean │ boolean │   uint64   │ int64 │ int64 │          double           │
├───────┼────────────────────┼────────────────────┼────────────────────┼──────────────┼──────────┼─────────┼────────────┼────────────┼─────────────┼─────────┼─────────┼────

In [53]:
from tqdm.notebook import tqdm
for it in tqdm(iterator):
    where_clause = " AND ".join([f"{partition_cols[i]} == {it[i]}" for i in range(len(it))])
    mean_variables = ", ".join([f"{fe_col}_avg_modis_median" for fe_col in fe_cols])
    join_statements = "\n".join([f"LEFT JOIN m_{fe_col} USING ({fe_col})" for fe_col in fe_cols])
    
    con.sql(f"""
        INSERT INTO design_matrix BY NAME (
            SELECT pixel_id, year, modis_median, {mean_variables}
                FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet')
                {join_statements}
                WHERE {where_clause}
        );
        """)
            

  0%|          | 0/17 [00:00<?, ?it/s]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Error: KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/traitlets/traitlets.py(708): __set__
  /scratch/ipykernel_1069269/1061293578.py(7): <module>
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/IPython/core/interactiveshell.py(3672): run_code
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/IPython/core/interactiveshell.py(3612): run_ast_nodes
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/IPython/core/interactiveshell.py(3367): run_cell_async
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/IPython/core/async_helpers.py(128): _pseudo_sync_runner
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/IPython/core/interactiveshell.py(3155): _run_cell
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/pytho

In [ ]:
from tqdm.notebook import tqdm
for fe_col in partition_cols.keys():
    if partition_cols[fe_col]:
        for it in tqdm(iterator):
            where_clause = " AND ".join([f"{partition_cols[fe_col][i]} == {it[i]}"for i in range(len(it))])
            
            con.sql(f"""
                CREATE OR REPLACE TEMP TABLE m1 AS
                SELECT {fe_col}, avg(modis_median) AS {fe_col}_avg_modis_median
                    FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet')
                    WHERE {where_clause}
                    GROUP BY {fe_col};
                    
                INSERT INTO design_matrix BY NAME (
                    SELECT pixel_id, year, modis_median, {fe_col}_avg_modis_median, ix, iy
                        FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet')
                            LEFT JOIN m1
                            USING (pixel_id)
                        WHERE {where_clause}
                );
                """)
            

  0%|          | 0/17 [00:00<?, ?it/s]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Error: KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/traitlets/traitlets.py(708): __set__
  /scratch/ipykernel_1069269/466942280.py(7): <module>
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/IPython/core/interactiveshell.py(3672): run_code
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/IPython/core/interactiveshell.py(3612): run_ast_nodes
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/IPython/core/interactiveshell.py(3367): run_cell_async
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/IPython/core/async_helpers.py(128): _pseudo_sync_runner
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python3.11/site-packages/IPython/core/interactiveshell.py(3155): _run_cell
  /scicore/home/meiera/schulz0022/miniforge-pypy3/envs/gnt/lib/python

In [ ]:
con.sql("SELECT avg(modis_median) OVER (PARTITION BY pixel_id) FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/modis.parquet/**/*.parquet') LIMIT 1000").show()

In [4]:
con.sql("""
        SELECT *
        FROM m1
        """).show()

┌─────────────────┬────────────────────┐
│    pixel_id     │  avg_modis_median  │
│     uint64      │       double       │
├─────────────────┼────────────────────┤
│     47244860234 │   259.562380952381 │
│     47244860519 │  258.2338095238095 │
│     47244863566 │ 260.02809523809526 │
│     47244863781 │  260.1257142857143 │
│     47244863900 │ 260.12761904761896 │
│     47244863955 │  259.7704761904762 │
│     47244864355 │ 258.67952380952374 │
│     47244864429 │ 259.12428571428575 │
│     47244865781 │  259.5266666666667 │
│     47244865923 │  260.1580952380952 │
│          ·      │          ·         │
│          ·      │          ·         │
│          ·      │          ·         │
│ 562992905044503 │  273.5495238095238 │
│ 562992905044568 │ 273.44047619047615 │
│ 562992905045043 │  273.1819047619048 │
│ 562992905045167 │  273.0904761904761 │
│ 562992905045236 │ 272.99142857142857 │
│ 562992905045363 │  273.1176190476191 │
│ 562992905045546 │ 273.13238095238097 │
│ 56299290504569

In [ ]:
con.sql("""
        CREATE OR REPLACE TABLE demeaned AS
            SELECT pixel_id, year, modis_median - m1.avg_modis_median
            FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/modis.parquet/**/*.parquet') d
                LEFT JOIN m1 USING (pixel_id)
        """)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
con.sql("""
        SELECT modis_median - avg(modis_median) OVER (PARTITION BY pixel_id) FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/modis.parquet/**/*.parquet')
        """).show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [ ]:
con.sql("SELECT avg(modis_median) OVER (PARTITION BY pixel_id) FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/modis.parquet/**/*.parquet') LIMIT 1000").show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [41]:
duckdb.read_parquet("/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet")

┌───────┬──────────────┬────────────┬────────────────┬──────────────┬──────────┬─────────┬────────────┬────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬──────────┐
│ year  │ modis_median │ modis_mean │ modis_rollmax3 │ viirs_annual │ ntl_harm │ reg_fav │ nb_mines_a │ nb_diamond │ subdivision │ country │ HDI_ME  │ HDI_VH  │  WB_HI  │  WB_LM  │ HDI_HI  │  WB_UM  │ pixel_id │
│ int16 │    double    │   double   │     double     │    double    │  double  │ boolean │   float    │   float    │    float    │  float  │ boolean │ boolean │ boolean │ boolean │ boolean │ boolean │  uint64  │
├───────┼──────────────┼────────────┼────────────────┼──────────────┼──────────┼─────────┼────────────┼────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼──────────┤
│  2000 │         NULL │       NULL │           NULL │         NULL │      0.0 │ false   │       NULL │       NULL │        NULL │    NULL │ false   │ f

In [28]:
con.sql("SELECT count() FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/modis.parquet/**/*.parquet')").show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│  17871105294 │
└──────────────┘



In [ ]:
con.sql("SELECT count() FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet') WHERE ntl_harm IS NOT NULL").show()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│  12201999306 │
└──────────────┘



In [11]:
con.sql("SELECT round(viirs_annual, 5), COUNT() FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/modis_subset.parquet/**/*.parquet') GROUP BY round(viirs_annual, 5)").show()
# ROUND_EVEN(modis_median, 5)

┌────────────────────────┬──────────────┐
│ round(viirs_annual, 5) │ count_star() │
│         double         │    int64     │
├────────────────────────┼──────────────┤
│                   0.31 │        39122 │
│                   0.52 │        25305 │
│                   0.11 │        24273 │
│                   0.46 │        29761 │
│                   1.24 │         5957 │
│                   1.09 │         7556 │
│                  16.04 │           40 │
│                   8.08 │          168 │
│                   2.04 │         2458 │
│                   1.19 │         6442 │
│                     ·  │            · │
│                     ·  │            · │
│                     ·  │            · │
│                  75.02 │            2 │
│                  22.55 │           35 │
│                  19.87 │           37 │
│                  53.12 │            6 │
│                  44.28 │           11 │
│                  50.74 │           10 │
│                  58.33 │        

In [13]:
con.sql("SELECT avg(modis_median) OVER (PARTITION BY pixel_id) FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/modis.parquet/**/*.parquet')").show()


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

: 

: 

: 

---

In [7]:
import duckdb

con = duckdb.connect("")
file = "read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/__modis.parquet/**/*.parquet')"

In [8]:
con.sql(
    f"""
    SELECT count(*)
    FROM {file}
    """
)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│  17825197929 │
└──────────────┘

In [9]:
con.sql(
    """
    SELECT *
    FROM read_parquet('/scicore/home/meiera/schulz0022/projects/growth-and-temperature/data_nobackup/assembled/_modis.parquet/**/*.parquet')
    """
)

┌───────┬──────────────┬────────────┬────────────────┬──────────────┬──────────┬─────────┬────────────┬────────────┬─────────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬─────────┬──────────┬───────┬───────┐
│ year  │ modis_median │ modis_mean │ modis_rollmax3 │ viirs_annual │ ntl_harm │ reg_fav │ nb_mines_a │ nb_diamond │ subdivision │ country │ HDI_ME  │ HDI_VH  │  WB_HI  │  WB_LM  │ HDI_HI  │  WB_UM  │ pixel_id │  ix   │  iy   │
│ int16 │    double    │   double   │     double     │    double    │  double  │ boolean │   float    │   float    │    float    │  float  │ boolean │ boolean │ boolean │ boolean │ boolean │ boolean │  uint64  │ int64 │ int64 │
├───────┼──────────────┼────────────┼────────────────┼──────────────┼──────────┼─────────┼────────────┼────────────┼─────────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼─────────┼──────────┼───────┼───────┤
│  2000 │         NULL │       NULL │           NULL │         NULL │      0.0 │ false  

17871105294, all columns, read problems
_ 17871105294, all columns
__ 17825197929